In [377]:
import pandas as pd
import numpy as np

Del siguiente listado con los autos autorizados para funcionar como taxi (https://www.nyc.gov/assets/tlc/images/content/pages/businesses/vehicles_approved_for_taxicabs.png), decidimos utilizar como referencia aquellos modelos que sigan disponibles, es decir, que no se encuenten discontinuados y cuyo ultimo modelo sea año 2023. El precio de referencia es el MSRP (manufacturer's suggested retail price) tambien conocido como precio de lista. El siguiente csv fue creado a partir de WebScrapping para obtener información sobre el precio de cada modelo, información extraída de la siguiente página: https://carbuzz.com/cars/

In [787]:
df_lista_autos = pd.read_csv("Datasets\Auxiliares\car_price.csv")

Importar distintos csv para buscar datos de consumo y de emisiones de CO2, cruzandolo con el dataset de modelos de autos. La fuente de donde se extrajeron los siguientes datasets es la siguiente: https://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64#wb-auto-6


Primer dataset: -My2023 Fuel Cosumption Ratings

In [788]:
#Importación del csv a un dataframe
df_fuel_ratings= pd.read_csv("Datasets\Fuel Consumption\MY2023 Fuel Consumption Ratings.csv")

In [789]:
#Selección de las columnas a utilizar
df_fuel_ratings= df_fuel_ratings[['Model Year','Make','Model','Fuel Type',
                                  'Fuel Consumption Comb (L/100 km)','CO2 Emissions (g/km)']]

In [790]:
def Obtener_info (dataframe):

    #Agregamos en una lista los modelos que queremos rastrear
    lista_modelos= df_lista_autos['Model'].tolist()
    list_indices= []

    #Evaluamos si se encuentra dicho modelo, de ser así agregamos el índice a una lista
    for index, value in enumerate (dataframe['Model']):
        for i in lista_modelos:
            if i.lower() in value.lower():
                list_indices.append(index)

    #Filtramos el dataframe según los índices obtenidos
    modelos= dataframe.iloc[list_indices]
    
    #Retornamos el dataframe filtrado
    return modelos

In [968]:
#Creamos el nuevo dataframe llamando a la función y tomamos los registros que necesitamos
modelos = Obtener_info(df_fuel_ratings).iloc[[0,1,8,10,13,15,17,20,22,26,30,32,33,37,38,44,46]]

#Reseteamos el índice para dejar ordenado el dataframe
modelos.reset_index(drop=True, inplace=True)

In [971]:
#Quitar palabras extra de los modelos para que coincidan con la lista de modelos
char_remov= ['Van', 'AWD', 'SE', 'LE', 'Cargo']

for index, value in enumerate (modelos['Model']):
    for char in char_remov:
        value = value.replace(char, "").strip()
        modelos.loc[index, 'Model'] = value

In [972]:
#Renombrar columnas para lograr la coincidencia entre dataframes
modelos.rename(columns= {'Fuel Consumption Comb (L/100 km)': 
                         'Consumption Comb (Le/100 km)'}, inplace=True)

Importar csv consumo de autos de bateria electrica Consumo\MY2012-2023 Battery Electric Vehicles.csv

In [807]:
#Importación del csv a un dataframe
df_consumo_electrico= pd.read_csv('Datasets\Fuel Consumption\MY2012-2023 Battery Electric Vehicles.csv')

In [808]:
#Selección de las columnas a utilizar
df_consumo_electrico= df_consumo_electrico[['Model Year','Make','Model', 'Fuel Type', 
                        'Consumption Comb (kWh/100 km)','Consumption Comb (Le/100 km)',
                        'CO2 Emissions(g/km)', 'Recharge Time(h)']]

In [973]:
#Creamos el nuevo dataframe llamando a la función y tomamos los registros que necesitamos
electricos= Obtener_info(df_consumo_electrico[df_consumo_electrico['Model Year'] 
                        == 2023]).iloc[[0,8,11,13,17]]

#Reseteamos el índice para dejar ordenado el dataframe
electricos.reset_index(drop=True, inplace=True)

In [977]:
#Quitar palabras extra de los modelos para que coincidan con la lista de modelos
char_remov= ['Standard Range', 'RWD']

for index, value in enumerate (electricos['Model']):
    for char in char_remov:
        value = value.replace(char, "").strip()
        electricos.loc[index, 'Model'] = value

In [978]:
#Renombrar columnas para lograr la coincidencia entre dataframes
electricos.rename(columns= {'CO2 Emissions(g/km)': 'CO2 Emissions (g/km)',
                            'Recharge Time(h)' : 'Recharge Time (h)'}, inplace=True)

In [979]:
#Acomodar nombre para que coincidan con la lista de modelos
electricos.loc[1, 'Model'] = 'Ioniq 5'

Importar csv consumo de autos de bateria electrica Consumo\MY2012-2023 Plug-in Hybrid Electric Vehicles.csv

In [838]:
#Importación del csv a un dataframe
df_consumo_hibrido= pd.read_csv('Datasets\Fuel Consumption\MY2012-2023 Plug-in Hybrid Electric Vehicles.csv')

In [839]:
#Selección de las columnas a utilizar
df_consumo_hibrido= df_consumo_hibrido[['Model Year','Make','Model','Fuel Type 1', 
                                        'Consumption Combined Le/100 km', 
                                        'Recharge Time (h)','CO2 Emissions (g/km)']]

In [980]:
#Creamos el nuevo dataframe llamando a la función y tomamos los registros que necesitamos
hibridos= Obtener_info(df_consumo_hibrido[df_consumo_hibrido['Model Year'] 
                       == '2023']).iloc[[0,1,2,5]]

#Reseteamos el índice para dejar ordenado el dataframe
hibridos.reset_index(drop=True, inplace=True)

In [981]:
#Renombrar registros para que coincidan con la lista de modelos
hibridos.loc[1, 'Model'] = 'Escape Hybrid'
hibridos.loc[3, 'Model'] = 'RAV4 Hybrid'

In [982]:
#Renombrar columnas para lograr la coincidencia entre dataframes
hibridos.rename(columns= {'Fuel Type 1' : 'Fuel Type', 'Consumption Combined Le/100 km' :
                          'Consumption Comb (Le/100 km)'}, inplace=True)

In [983]:
#Crear nueva columna
hibridos['Consumption Comb (kWh/100 km)'] = 0

#Asignar valores en diferentes columnas según corresponda
for index, value in enumerate (hibridos['Consumption Comb (Le/100 km)']):
    hibridos.loc[index, 'Consumption Comb (kWh/100 km)'] = value[6:10]
    hibridos.loc[index, 'Consumption Comb (Le/100 km)'] = value[:3]

Creación dataframe con toda la informacion de los autos, el precio y la contaminacion

In [984]:
#Concatenación de los distintos tipos de autos
df_cars= pd.concat([modelos, electricos, hibridos], ignore_index=True)

#Ordenar el dataframe según la marca y el modelo
df_cars.sort_values(['Make', 'Model'], ignore_index=True, inplace=True)

#Mergear el precio de los autos en función del modelo
df_cars= df_cars.merge(df_lista_autos[['MSRP(usd)','Model']], on='Model')

In [985]:
#Creación de un Id por cada modelo de auto
id_autos = np.arange(1, 27)
df_cars.insert(0, 'Vehicle_id', id_autos)

Importar dataset de contaminación sonora, extraído del siguiente link:
 
https://www.kaggle.com/datasets/murtio/car-noise-specification

In [ ]:
#Importación de dataset
df_noise = pd.read_csv('Datasets\Auxiliares\car_noise_specification_datasets.csv')
#Selección de columnas a utilizar
df_noise= df_noise[['brand', 'model', 'year', 'spec','dB_at_80kmh']]
#Eliminar columnas con valores nulos
df_noise.dropna(inplace=True)
#Renombrar la columna para usar la función
df_noise.rename(columns={'model' : 'Model'}, inplace=True)

In [987]:
#Extraer información de contaminación sonora según los modelos de autos
df_noise= Obtener_info(df_noise).iloc[[0,3,4,7,8,13,14,15,17,24,37,38,39,40]]
#Resetear index para ordenar el dataframe
df_noise.reset_index(drop=True, inplace=True)

In [988]:
#Traspasar el nombres 'Hybrid' de la columna spec a la columna Model
condicion = df_noise['spec'] == 'Hybrid'

df_noise.loc[condicion, 'Model'] = (df_noise.loc[condicion, 'Model'] + ' ' +
                                    df_noise.loc[condicion, 'spec'])

In [989]:
#Seleccionar columnas a utilizar
df_noise = df_noise[['brand', 'Model', 'dB_at_80kmh']]

In [990]:
#Agregar al df_cars la información de df_noise según cada modelo
df_cars= df_cars.merge(df_noise[['dB_at_80kmh', 'Model']], on= 'Model', how= 'left')

In [991]:
#Calcular valores promedios de ruido para aquellos modelos que no tienen información,
#discriminando según el tipo de modelo (híbrido o combustión)
media_hibrido= df_noise.iloc[[3,6,7,11]]['dB_at_80kmh'].mean()
media_combustion= df_noise.iloc[[0,1,2,4,5,8,9,10,12,13]]['dB_at_80kmh'].mean()

In [992]:
#Agregar promedio a modelos híbridos
registros= [1,3,11]
df_cars.loc[registros, 'dB_at_80kmh'] = media_hibrido

#Agregar promedio a modelos a combustión
registros= [5,12,16,18]
df_cars.loc[registros, 'dB_at_80kmh'] = media_combustion

In [993]:
#Agregar nivel de ruido de modelos eléctricos, obtenido de: 
#https://tbtp-ev.github.io/tbtp-results-noise.html

df_cars.loc[6, 'dB_at_80kmh'] = 65.7
df_cars.loc[14, 'dB_at_80kmh'] = 67.2
df_cars.loc[10, 'dB_at_80kmh'] = 64.8
df_cars.loc[4, 'dB_at_80kmh'] = 65.8
df_cars.loc[15, 'dB_at_80kmh'] = 65.8

In [ ]:
#Seleccionar columnas para df_car_models
df_car_models= df_cars[['Vehicle_id', 'Model Year', 'Make', 'Model', 'MSRP(usd)']]
#Ordenar el dataframe según el Vehicle_id
df_car_models.sort_values('Vehicle_id', inplace=True)
#Exportar a csv
df_car_models.to_csv('Datasets\Finales\car_models.csv', index=False)

In [ ]:
#Seleccionar columnas para df_pollution
df_pollution = df_cars[['Vehicle_id', 'Fuel Type','Consumption Comb (Le/100 km)', 
                         'Consumption Comb (kWh/100 km)','Recharge Time (h)', 
                         'CO2 Emissions (g/km)', 'dB_at_80kmh' ]]
#Ordenar el dataframe según Fuel Tipe
df_pollution.sort_values('Fuel Type', inplace=True)
#Exportar a csv
df_pollution.to_csv('Datasets\Finales\pollution.csv', index=False)